<a href="https://colab.research.google.com/github/domino4com/Parklands-ASRI/blob/main/Parklands_ASRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from scipy.integrate import cumtrapz

# Step 1: Load the CSV file
# Replace 'rocket_data.csv' with your actual filename
df = pd.read_csv('rocket_data.csv')

# Step 2: Extract relevant columns
time_ms = df['T-Zero Time [ms]']  # Time since launch in milliseconds
time_s = time_ms / 1000.0         # Convert to seconds
ax_g = df['X [g]']                # Acceleration in X (assumed North)
ay_g = df['Y [g]']                # Acceleration in Y (assumed East)
altitude_m = df['Altitude [m]']   # Altitude in meters

# Step 3: Convert accelerations from g to m/s²
g = 9.81
ax = ax_g * g  # X acceleration in m/s²
ay = ay_g * g  # Y acceleration in m/s²

# Step 4: Integrate accelerations to get velocities and positions
vx = cumtrapz(ax, time_s, initial=0)  # Velocity in X (m/s)
vy = cumtrapz(ay, time_s, initial=0)  # Velocity in Y (m/s)
x = cumtrapz(vx, time_s, initial=0)   # North displacement (meters)
y = cumtrapz(vy, time_s, initial=0)   # East displacement (meters)

# Step 5: Convert displacements to latitude and longitude
R_earth = 6371000  # Earth's radius in meters
start_lat = -34.602642  # Starting latitude in degrees
start_lon = 20.302401   # Starting longitude in degrees

delta_lat = (x / R_earth) * (180 / np.pi)
delta_lon = (y / (R_earth * np.cos(np.radians(start_lat)))) * (180 / np.pi)
latitude = start_lat + delta_lat
longitude = start_lon + delta_lon

# Step 6: Reduce waypoints to ~10% by sampling every 10th row
skip = 10
latitude_reduced = latitude[::skip]
longitude_reduced = longitude[::skip]
altitude_reduced = altitude_m[::skip]

# Step 7: Generate KML file with altitude parameters
kml_header = '''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document>
    <Placemark>
      <LineString>
        <extrude>1</extrude>
        <altitudeMode>absolute</altitudeMode>
        <coordinates>'''
kml_footer = '''        </coordinates>
      </LineString>
    </Placemark>
  </Document>
</kml>'''

# Create coordinates string in the format: longitude,latitude,altitude
coordinates = []
for lon, lat, alt in zip(longitude_reduced, latitude_reduced, altitude_reduced):
    coordinates.append(f"{lon},{lat},{alt}")
coordinates_str = "\n".join(coordinates)

# Write to KML file
with open('rocket_flight.kml', 'w') as f:
    f.write(kml_header)
    f.write(coordinates_str)
    f.write(kml_footer)

print("KML file 'rocket_flight.kml' generated successfully!")

KML file 'rocket_flight.kml' generated successfully!


<ipython-input-1-c1d802c598cb>:23: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  vx = cumtrapz(ax, time_s, initial=0)  # Velocity in X (m/s)
<ipython-input-1-c1d802c598cb>:24: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  vy = cumtrapz(ay, time_s, initial=0)  # Velocity in Y (m/s)
<ipython-input-1-c1d802c598cb>:25: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  x = cumtrapz(vx, time_s, initial=0)   # North displacement (meters)
<ipython-input-1-c1d802c598cb>:26: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  y = cumtrapz(vy, time_s, initial=0)   # East displacement (meters)
